In [9]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
!pip install mlflow

     |████████████████████████████████| 17.9 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 209 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 497 kB/s eta 0:00:01
     |████████████████████████████████| 95 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 407 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 773 kB/s eta 0:00:01
     |████████████████████████████████| 156 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 2.1 MB

In [10]:
data = pd.read_csv("data/wine-quality.csv")

In [11]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [15]:
# Tracking Methods(default storage)

remote_server_uri = "http://0.0.0.0:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [16]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [17]:
exp_name = "ElasticNet_Wine"
mlflow.set_experiment(exp_name)

2022/06/30 15:05:46 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_Wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_Wine', tags={}>

In [18]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y

def train(alpha=0.5, l1_ratio=0.5):
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)
    
    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")


In [19]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666
Save to: mlruns/1/d800605d39854dda8b92aa4718243ed1/artifacts


In [20]:
train(0.8, 0.8)

Elasticnet model (alpha=0.800000, l1_ratio=0.800000):
  RMSE: 0.8598685637629909
  MAE: 0.6478991380829888
  R2: 0.045042561953817395
Save to: mlruns/1/0d04246a978c48719bb520ca0e5f3efa/artifacts


In [23]:
# Tagging Runs

from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>, <Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_Wine', tags={}>]


In [25]:
_run = client.get_run(run_id="0d04246a978c48719bb520ca0e5f3efa")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6478991380829888,
 'r2': 0.045042561953817395,
 'rmse': 0.8598685637629909}, params={'alpha': '0.8', 'l1_ratio': '0.8'}, tags={'mlflow.log-model.history': '[{"run_id": "0d04246a978c48719bb520ca0e5f3efa", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2022-06-30 09:43:35.728320", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"loader_module": "mlflow.sklearn", '
                             '"python_version": "3.8.12", "env": '
                             '"conda.yaml"}, "sklearn": {"pickled_model": '
                             '"model.pkl", "sklearn_version": "1.0.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"956b33faa76b406395da2f3d736a6c7e", '
                             '"mlflow_ve

In [26]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)